In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

c:\Users\gupta\Desktop\Virtual Assistant\walmart-assistant-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the .env file

load_dotenv()

True

In [3]:
# Fetch API key
api_key = os.getenv("GOOGLE_API_KEY")

In [4]:
# Configure Gemini
genai.configure(api_key=api_key)

In [5]:
# Initialize the model
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
# Simple test prompt
response = model.generate_content("What are the best laptops under ₹50,000 for students?")
print(response.text)

In [9]:
user_state = {
    "category": None,
    "budget": None,
    "use_case": None,
    "brand_preferences": [],
    "important_features": [],
    "excluded_features": [],
    "suggested_products": [],
    "chat_history": []
}

In [10]:
user_input = "I want a phone for basic usage such as camera and basic calling and busget is 150$"

In [11]:
# ...existing code...
prompt = f"""
IMPORTANT: You MUST always explain every technical term or jargon you use, immediately after mentioning it, in simple language. 
For example: "This is a top load washing machine (which means you load clothes from the top)."

You are a smart shopping assistant for Walmart.
The user said: "{user_input}"

Their current preferences:
- Category: {user_state['category']}
- Budget: {user_state['budget']}
- Use case: {user_state['use_case']}
- Preferred brands: {', '.join(user_state['brand_preferences'])}
- Important features: {', '.join(user_state['important_features'])}

Based on this, do one of the following:
1. Ask a clarifying question if needed.
2. Recommend products (example-based).
3. Explain any technical terms you use.
4. Update their preferences if something new was mentioned.

Respond like a helpful assistant.
"""
# ...existing code...

In [12]:
#Getting response from the model
response = model.generate_content(prompt)
assistant_reply = response.text
print(f"\n🛍️ Assistant:\n{response.text}")


🛍️ Assistant:
Okay, a phone for basic calling and camera use with a $150 budget.  That's helpful! To give you the best recommendations, I need a little more information.

Do you have a preference for a particular operating system (OS), like Android or iOS (Apple)?  An OS is basically the software that runs the phone and determines how you interact with it. Android is known for its open-source nature and customization options, while iOS is known for its simplicity and user-friendliness.

Once I know your OS preference (or if you don't have one!), I can start suggesting some great Walmart options within your price range.



In [13]:
# Update user state with the assistant's reply
user_state["chat_history"].append({
    "user": user_input,
    "assistant": assistant_reply
})

In [14]:
# Using the state extraction prompt to extract new preferences
# This is a separate prompt to extract any new preferences from the user input and assistant reply
state_extraction_prompt = f"""
You are an intelligent state extraction system.
Based on the user input and assistant reply, extract any new preferences.

User: "{user_input}"
Assistant: "{assistant_reply}"

Return only the updated fields in this JSON format:
{{
  "budget": "...",
  "use_case": "...",
  "category": "...",
  "brand_preferences": ["..."],
  "important_features": ["..."],
  "excluded_features": ["..."]
}}
If nothing was mentioned, return an empty JSON object: {{}}
"""

In [18]:
import json
state_response = model.generate_content(state_extraction_prompt)

import re
raw = state_response.text
match = re.search(r'\{.*\}', raw, re.DOTALL)
if match:
    extracted = json.loads(match.group())
    # Update user_state with extracted values
    for key, value in extracted.items():
        if key in user_state:
            user_state[key] = value
else:
    print("No JSON found in response.")

In [19]:
# Print the updated user state for verification

print("\n📦 Updated User State:")
print(json.dumps(user_state, indent=2))


📦 Updated User State:
{
  "category": "phone",
  "budget": "150$",
  "use_case": "basic calling and camera use",
  "brand_preferences": [],
  "important_features": [
    "camera",
    "basic calling"
  ],
  "excluded_features": [],
  "suggested_products": [],
  "chat_history": [
    {
      "user": "I want a phone for basic usage such as camera and basic calling and busget is 150$",
      "assistant": "Okay, a phone for basic calling and camera use with a $150 budget.  That's helpful! To give you the best recommendations, I need a little more information.\n\nDo you have a preference for a particular operating system (OS), like Android or iOS (Apple)?  An OS is basically the software that runs the phone and determines how you interact with it. Android is known for its open-source nature and customization options, while iOS is known for its simplicity and user-friendliness.\n\nOnce I know your OS preference (or if you don't have one!), I can start suggesting some great Walmart options w

In [ ]:
import re
import json

while True:
    user_input = input("👤 You: ")
    if user_input.strip().lower() in ["stop", "exit", "quit"]:
        print("👋 Conversation ended.")
        break

    # Build prompt with current state
    prompt = f"""
IMPORTANT: You MUST always explain every technical term or jargon you use, immediately after mentioning it, in simple language.
For example: "This is a top load washing machine (which means you load clothes from the top)."

You are a smart shopping assistant for Walmart.
The user said: "{user_input}"

Their current preferences:
- Category: {user_state['category']}
- Budget: {user_state['budget']}
- Use case: {user_state['use_case']}
- Preferred brands: {', '.join(user_state['brand_preferences'])}
- Important features: {', '.join(user_state['important_features'])}

Based on this, do one of the following:
1. Ask a clarifying question if needed.
2. Recommend products (example-based).
3. Explain any technical terms you use.
4. Update their preferences if something new was mentioned.

Respond like a helpful assistant.
"""

    response = model.generate_content(prompt)
    assistant_reply = response.text
    print(f"\n🛍️ Assistant:\n{assistant_reply}")

    # Update chat history
    user_state["chat_history"].append({
        "user": user_input,
        "assistant": assistant_reply
    })

    # Extract new preferences from the conversation
    state_extraction_prompt = f"""
You are an intelligent state extraction system.
Based on the user input and assistant reply, extract any new preferences.

User: "{user_input}"
Assistant: "{assistant_reply}"

Return only the updated fields in this JSON format:
{{
  "budget": "...",
  "use_case": "...",
  "category": "...",
  "brand_preferences": ["..."],
  "important_features": ["..."],
  "excluded_features": ["..."]
}}
If nothing was mentioned, return an empty JSON object: {{}}
"""
    state_response = model.generate_content(state_extraction_prompt)
    raw = state_response.text
    match = re.search(r'\{.*\}', raw, re.DOTALL)
    if match:
        extracted = json.loads(match.group())
        for key, value in extracted.items():
            if key in user_state:
                user_state[key] = value
    else:
        print("No JSON found in response.")

    # Optionally, print updated state for debugging
    # print("\n📦 Updated User State:")
    # print(json.dumps(user_state, indent=2))